## EDA of larger dataset

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import sys

pd.set_option("display.max_rows", 10)

In [2]:
messages_df = pd.read_csv("../NEO-RCT-labeled/mwach-dump-2022-04-05/mwbase_message-2022-04-05.csv")

#Remove messages with no participants -- spam
messages_df = messages_df[messages_df.participant_id.notnull()]

#Sort by participant and date
messages_df = messages_df.sort_values(by=['participant_id', 'created'], ascending=[True, True])


c:\python38\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (15,24) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [3]:
### Generate sent_by and select variables of importance
messages_df['sent_by'] = np.where(messages_df.System == 1, 'system', 
                                  np.where((messages_df.Out == 1) & (messages_df.System == 0), 'nurse', 'participant'))

messages_final = messages_df[['ID','created','Out','System','languages','participant_id', 'Message Urgency','sent_by', 'text']]

#rename urgency column
messages_final = messages_final.rename(columns={'Message Urgency':'Urgency'})

#create label variable
messages_final['label'] = np.where(((messages_final.Urgency == 1) | (messages_final.Urgency == 2)), 1,
                                  np.where((messages_final.Urgency == 3) | (messages_final.Urgency == 4) | (messages_final.Urgency == 5), 0, -2))


In [ ]:
#info
messages_final

In [ ]:

#!{sys.executable} -m pip install pandas-profiling
#Describe
#messages_df.describe()

In [ ]:
#!{sys.executable} -m pip install --upgrade IProgress
#!jupyter nbextension enable --py widgetsnbextension

In [4]:
messages_final['Urgency'].value_counts(dropna=False)

-2    159454
 5      3682
 3      2334
 2      1786
 4      1399
 1       451
Name: Urgency, dtype: int64

In [5]:
messages_df['System'].value_counts(dropna=False)

1    93160
0    75946
Name: System, dtype: int64

In [6]:
messages_df.Out.value_counts(dropna=False)

1    122443
0     46663
Name: Out, dtype: int64

In [7]:
messages_df.sent_by.value_counts(dropna=False)

system         93160
participant    46663
nurse          29283
Name: sent_by, dtype: int64

In [8]:
messages_final.label.value_counts(dropna=False)

-2    159454
 0      7415
 1      2237
Name: label, dtype: int64

In [ ]:
#Save raw dataset
messages_final.to_csv("Data/messages_df.csv")

### Data Cleaning


In [9]:
#generate json of names for removing in message texts
participant =  pd.read_csv("../NEO-RCT-labeled/mwach-dump-2022-04-05/mwneo_participant-2022-04-05.csv")

json_string = participant['SMS Name'].to_json(orient='values')
# Using a JSON string
with open('Data/names2.json', 'w') as outfile:
    outfile.write(json_string)

# Functions
def removeNames(df, text_column, json_file):
    # Opening JSON file
    f = open(json_file,)
    # returns JSON object
    names = json.load(f)

    p = re.compile('|'.join(map(re.escape, names)))
    df[text_column] = [p.sub('', str(text)) for text in df[text_column]]

    return df
    
def removeSalutation(row, text_column):
    ''' #Remove salutation '''
    sent = str(row[text_column])
    result = re.match(r".+, this is \b[A-Z][a-z]+ from \b[A-Z][a-z]+", sent)
    result_swahili = re.match(r".+, huyu ni \b[A-Z][a-z]+ kutoka \b[A-Z][a-z]+", sent)
    result_luo = re.match(r".+, mae en \b[A-Z][a-z]+ mawuok \b[A-Z][a-z]+", sent)
    result_welcome = re.match(r".+Welcome to Mobile WACh NEO \b[A-Z][a-z]+", sent)
    result_karibu = re.match(r".+Karibu kwa Mobile WACh NEO \b[A-Z][a-z]+", sent)
    #mobile WACh NEO
    result_wachneo = re.match(r".+ mobile WACh NEO \b[A-Z][a-z]+", sent)
    result_wachcaps = re.match(r".+ Mobile WACh NEO \b[A-Z][a-z]+", sent)

    if(result):
        sentence = sent.split(str(result.group()),1)[1]
    elif(result_swahili):
        sentence = sent.split(str(result_swahili.group()),1)[1]
    elif(result_luo):
        sentence = sent.split(str(result_luo.group()),1)[1]
    elif(result_wachneo):
        sentence = sent.split(str(result_wachneo.group()),1)[1]
    elif(result_welcome):
        sentence = sent.split(str(result_welcome.group()),1)[1]
    elif(result_karibu):
        sentence = sent.split(str(result_karibu.group()),1)[1]
    elif(result_wachcaps):
        sentence = sent.split(str(result_wachcaps.group()),1)[1]
    else:
        sentence = sent

    return sentence

def removePunctuations(row, column):
    texts = str(row[column])
    #print(texts)
    for punctuation in string.punctuation:
        #print(punctuation)
        texts = texts.replace(punctuation, '')

    return texts

In [ ]:
participant['SMS Name'].to_csv("Data/participant_names2.csv")

In [10]:
import json
import re
import string 

#1. remove salutations
text_column = "text"
maindf = messages_final
maindf = maindf.assign(text = maindf.apply(lambda row: removeSalutation(row, text_column), axis=1))

#2. remove punctuation marks
text_column = "text"
maindf = maindf.assign(text = maindf.apply(lambda row: removePunctuations(row, text_column), axis=1))

#3. remove names
maindf = removeNames(maindf, 'text', "Data/names.json")
maindf = removeNames(maindf, 'text', "Data/names2.json")

#4. remove validation messages
#maindf = maindf[maindf['topic'] != "validation"]

maindf_short = maindf[['ID','created','Out','System','languages','participant_id', 'Urgency','sent_by', 'text', 'label']]
maindf_short = maindf_short.reset_index()

#Write to current folder
maindf_short.to_csv("Data/main_df_v4.csv")

#5. Generate context_id for System context
maindf_short['context_id'] = 0
context_id = 0
i = 0
for row in maindf_short.itertuples():
    if(row.sent_by == "system"):
        context_id = row.ID
    #maindf_short.set_value(i, "context_id", context_id)
    maindf_short.at[i, 'context_id'] = context_id
    i += 1

#6. System messages
system_df = maindf_short[maindf_short['sent_by'] == 'system']
system_df = system_df[["ID", "text"]]
system_df['system_text'] = system_df.text
system_df = system_df[["ID", "system_text"]]


In [11]:
#7. Participants messages - System Context
participant_df = maindf_short[maindf_short['sent_by'] == 'participant']
#Participants - add system context
part_context = pd.merge(participant_df, system_df, how="left", left_on="context_id", right_on="ID")
#dealing with all messages, including those without preceding context
part_context['contextualized'] = np.where(part_context.context_id == 0, part_context.text, part_context.system_text+" "+part_context.text)

participant_context = part_context[['ID_x', 'participant_id', 'contextualized', 'label']]
participant_context['mid'] = participant_context.ID_x
participant_context['text'] = participant_context.contextualized
participant_context = participant_context[['mid', 'participant_id', 'text', 'label']]

#get unlabeled system_context messages
system_context_unlbl = participant_context[participant_context.label < 0]
system_context_unlbl.to_csv("Data/system_context_all_unlbl_v4.csv")

#get labeled system_context messages
system_context_lbl = participant_context[participant_context.label >= 0]
system_context_lbl.to_csv("Data/system_context_all_lbl_v4.csv")

#System Context train test split
train_df, remain_df = train_test_split(system_context_lbl, random_state=42, train_size=0.7, stratify=system_context_lbl.label.values)
test_df, dev_df = train_test_split(remain_df, random_state=42, train_size=0.7, stratify=remain_df.label.values)

train_df.to_csv("Data/train_df_v4.csv")
test_df.to_csv("Data/test_df_v4.csv")
dev_df.to_csv("Data/dev_df_v4.csv")

#formulate system context pretraining dataset
system_context_pretrain_df = pd.concat([system_context_unlbl, train_df])
system_context_pretrain_df.to_csv("Data/system_context_pretrain_v4.csv")
system_pretrain_train, system_pretrain_validation = train_test_split(system_context_pretrain_df, test_size=0.2, random_state=42)
system_pretrain_train.text.to_csv("Data/system_pretrain_train_v4.csv")
system_pretrain_validation.text.to_csv("Data/system_pretrain_dev_v4.csv")

<ipython-input-11-a1c938dea95c>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  participant_context['mid'] = participant_context.ID_x
<ipython-input-11-a1c938dea95c>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  participant_context['text'] = participant_context.contextualized
